In [29]:
import pandas as pd
from pymongo import MongoClient

In [30]:
client = MongoClient()
db = client.nobel_prize
cursor = db.winners.find()
df = pd.DataFrame(list(cursor))

In [31]:
df

,_id,catagory,name,nationality,sex,year
0,59a552afb6435fc01cac169a,Physics,Albert Einstein,Swiss,male,1921
1,59a552afb6435fc01cac169b,Physics,Paul Dirac,British,male,1933
2,59a552afb6435fc01cac169c,Chemistry,Marie Curie,Polish,female,1911


In [32]:
records = df.to_dict('records')

In [33]:
records

[{'_id': ObjectId('59a552afb6435fc01cac169a'),
  'catagory': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'_id': ObjectId('59a552afb6435fc01cac169b'),
  'catagory': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'_id': ObjectId('59a552afb6435fc01cac169c'),
  'catagory': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [34]:
from pymongo import MongoClient
DB_NOBEL_PRIZE = 'nobel_prize'
COLL_WINNERS = 'winners'

In [35]:
def get_mongo_database(db_name, host='localhost', port=27017, username=None, password=None):
    if username and password:
        mongo_uri = 'mongodb://{}:{}@{}/()'.format(username, password, host, db_name)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
    
    return conn[db_name]

In [36]:
def mongo_to_dataframe(db_name, collection, query=None, host='localhost', port=27017,
                       username=None, password=None, no_id=True):
    db = get_mongo_database(db_name, host, port, username, password)
    
    if query == None:
        query = {}
    cursor = db[collection].find(query)
    df = pd.DataFrame(list(cursor))
    
    if no_id:
        del df['_id']
        
    return df

In [37]:
def dataframe_to_mongo(df, db_name, collection, host='localhost', port=27017,
                       username=None, password=None):
    db = get_mongo_database(db_name, host, port, username, password)
    
    records = df.to_dict('records')
    db[collection].insert_many(records)

In [38]:
df = mongo_to_dataframe(DB_NOBEL_PRIZE, COLL_WINNERS)
df

,catagory,name,nationality,sex,year
0,Physics,Albert Einstein,Swiss,male,1921
1,Physics,Paul Dirac,British,male,1933
2,Chemistry,Marie Curie,Polish,female,1911


In [40]:
dataframe_to_mongo(df, DB_NOBEL_PRIZE, COLL_WINNERS + "a")

In [41]:
df = mongo_to_dataframe(DB_NOBEL_PRIZE, COLL_WINNERS, no_id=False)
df

,_id,catagory,name,nationality,sex,year
0,59a552afb6435fc01cac169a,Physics,Albert Einstein,Swiss,male,1921
1,59a552afb6435fc01cac169b,Physics,Paul Dirac,British,male,1933
2,59a552afb6435fc01cac169c,Chemistry,Marie Curie,Polish,female,1911


In [42]:
dataframe_to_mongo(df, DB_NOBEL_PRIZE, COLL_WINNERS + "b")

In [43]:
df = pd.read_json('/Users/simonsu/git/dataviz/scrapy/nobel_winners/nobel_full.json')

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1065 entries, 0 to 1064
Data columns (total 12 columns):
born_in           1065 non-null object
catagory          1065 non-null object
country           1065 non-null object
date_of_birth     1056 non-null object
date_of_death     709 non-null object
gender            1056 non-null object
link              1065 non-null object
name              1065 non-null object
place_of_birth    1056 non-null object
place_of_death    709 non-null object
text              1065 non-null object
year              1065 non-null int64
dtypes: int64(1), object(11)
memory usage: 108.2+ KB


In [45]:
dataframe_to_mongo(df, DB_NOBEL_PRIZE, COLL_WINNERS)

In [46]:
df.describe()

,year
count,1065.000000
mean,1968.008451
std,69.116735
min,0.000000
25%,1948.000000
50%,1976.000000
75%,1998.000000
max,2016.000000
